In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

In [2]:
import psycopg2
import pandas as pd
# Specify the file path
file_path = "C:/Users/USER/OneDrive/Documents/studentmarks.xlsx"
# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path)

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df

,Student Name,Marks
0,Alice,0.84
1,Bob,0.57
2,Carol,0.95
3,Emily,0.69
4,Frank,0.41
5,Henry,0.74
6,Jack,0.33


In [5]:
data = { 
         "first name" : ["John", "James", "Mary"], 
         "Age" :  [22, 34, 30], 
         "GENDER":["M", "M", "F"], 
         "salary ":[30000, 56000, 44000]
       }

In [6]:
mydf = pd.DataFrame(data)

In [7]:
type(data)

dict

In [8]:
mydf.columns

Index(['first name', 'Age', 'GENDER', 'salary '], dtype='object')

In [9]:
mydf['Age']

0    22
1    34
2    30
Name: Age, dtype: int64

In [10]:
def cleanColumn(data):
    #get all the key values
    keysList = list(data.keys())
    
    #loop through the key values
    for key in keysList:
        #replace the uppercase key values with lowercase values
        data[key.lower()] = data.pop(key)
        
    #remove the white spaces
    data = { x.translate({32:None}) : y 
                 for x, y in data.items()}
    return data
#run the map        
# map(cleanColumn(data), data)

#run the clean column function
new_data = cleanColumn(data)

#display the new key values
new_data

{'firstname': ['John', 'James', 'Mary'],
 'age': [22, 34, 30],
 'gender': ['M', 'M', 'F'],
 'salary': [30000, 56000, 44000]}

In [11]:
conn = psycopg2.connect("dbname=myduka user=postgres password=Candidcoco@20")

In [12]:
df = pd.read_sql_query("SELECT sales.id as sales_id, product_name, buying_price, selling_price, sales.quantity as sales_quantity, created_at as sales_created_at FROM products,sales WHERE products.id = sales.pid", conn) 

In [13]:
df.set_index('sales_id', inplace=True,verify_integrity=False)

In [14]:
df['profit'] = df['selling_price'] - df['buying_price']

In [15]:
df['new_selling_price'] = df['buying_price'] * 0.25

In [16]:
df['new_selling_price'] = df['new_selling_price'] * 10

In [17]:
df['new_profit'] = (df['new_selling_price'] - df['buying_price']) * df['sales_quantity']

In [18]:
df.drop(columns=['selling_price', 'profit'], inplace=True)

In [19]:
column_names = {'new_selling_price': 'selling_price', 'new_profit': 'profit'}
df.rename(columns=column_names, inplace=True)

In [20]:
df.head(5)

,product_name,buying_price,sales_quantity,sales_created_at,selling_price,profit
sales_id,,,,,,
1,Vinegar - Tarragon,65,23,2022-11-13,162.5,2242.5
2,Cloves - Whole,33,12,2022-08-31,82.5,594.0
3,"Pail - 15l White, With Handle",47,92,2022-12-29,117.5,6486.0
4,"Chocolate - Pistoles, Lactee, Milk",75,17,2023-04-08,187.5,1912.5
5,Cassis,16,100,2023-05-25,40.0,2400.0


In [21]:
X = df[['selling_price','sales_quantity']]
y = df['profit'] 

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 104, test_size=0.25, shuffle = True)

In [23]:
regr = LinearRegression()
# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression()

In [24]:
# Make predictions using the testing set
y_test_pred = regr.predict(X_test)
y_train_pred = regr.predict(X_train)

In [25]:
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 957852.919, test: 1824659.372
R^2 train: 0.899, test: 0.840
